In [4]:
'''
Author: Ngo Van Uc
Date: 31/05/2024
'''

'\nAuthor: Ngo Van Uc\nDate: 31/05/2024\n'

***tasks:***

- mở file getLinkProduct_PhoneDevices.csv
- đọc thành frame
- loại bỏ "utm_flashsale=1" trong link sản phẩm
- xử lí các data trùng nhau
- mở link và lấy các comment của sản phẩm trên trang đó

In [5]:
import os
import csv
import time
import re
import random
import numpy as np
import pandas as pd
from multiprocessing import Process, Queue
from urllib.request import urlopen
from urllib.request import Request
from urllib.error import HTTPError
from urllib.error import URLError
from bs4 import BeautifulSoup

In [6]:
# User agent strings
user_agents = [
    'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36',
]

In [7]:
file_path = "E:/UDA_LEARNING/UDA_XU LI NGON NGU TU NHIEN II/DienMayXanhCrawling/data/getLinkProduct_PhoneDevices.csv"
df = pd.read_csv(file_path)
df.head()

,data-id,data-productcode,data-price,href,data-name,data-id .1,data-price .1,data-brand,data-cate,item-rating-total
0,305659,131491003841,40990000.0,https://www.dienmayxanh.com/dien-thoai/iphone-...,Điện thoại iPhone 15 Pro Max 512GB ...,305659,37490000.0,iPhone (Apple),Điện thoại,24
1,299033,131491003814,28990000.0,https://www.dienmayxanh.com/dien-thoai/iphone-...,Điện thoại iPhone 15 Pro 128GB ...,299033,25990000.0,iPhone (Apple),Điện thoại,17
2,303891,131491003803,25990000.0,https://www.dienmayxanh.com/dien-thoai/iphone-...,Điện thoại iPhone 15 Plus 128GB ...,303891,23490000.0,iPhone (Apple),Điện thoại,27
3,281570,131491003786,22990000.0,https://www.dienmayxanh.com/dien-thoai/iphone-...,Điện thoại iPhone 15 128GB ...,281570,19990000.0,iPhone (Apple),Điện thoại,29
4,319665,131491003957,22990000.0,https://www.dienmayxanh.com/dien-thoai/samsung...,Điện thoại Samsung Galaxy S24 5G 256GB ...,319665,21990000.0,Samsung,Điện thoại,6


In [8]:
# loại bỏ các dữ liệu có data_id trùng nhau
print(len(df))
uni_df = df.drop_duplicates().reset_index(drop=True)
print(len(uni_df))

315
168


In [9]:
# loại bỏ tham số trong đường link "?utm_flashsale=1" 
# thêm "/danh-gia" vào đường link và đưa vào data frame mới
# Ex: https://www.dienmayxanh.com/dien-thoai/samsung-galaxy-s24-5g?utm_flashsale=1

def removeArgLink(link):
    link1 = "https://www.dienmayxanh.com/dien-thoai/iphone-15-pro?utm_flashsale=1                     "
    link2 = "https://www.dienmayxanh.com/dien-thoai/iphone-15-pro                     "
    link3 = "https://www.dienmayxanh.com/dien-thoai/iphone-15-pro"
    regular = r'\?utm_flashsale=1*'
    newLink = re.sub(regular, '', link)
    regular = r'\ *'
    newLink = re.sub(regular, '', newLink)
    newLink += '/danh-gia'
    return newLink

link1 = "https://www.dienmayxanh.com/dien-thoai/iphone-15-pro?utm_flashsale=1                     "
link2 = "https://www.dienmayxanh.com/dien-thoai/iphone-15-pro                     "
link3 = "https://www.dienmayxanh.com/dien-thoai/iphone-15-pro"
# print(removeArgLink(str(uni_df.iloc[0, 3])))
# print(str(uni_df.iloc[1, 3]))

# run this code only onetime
for idx in range(0, len(uni_df)):
    uni_df.iloc[idx, 3] = removeArgLink(uni_df.iloc[idx, 3])

In [10]:
print(uni_df.iloc[0, 3])

https://www.dienmayxanh.com/dien-thoai/iphone-15-pro-max-512gb/danh-gia


In [11]:
print(str(uni_df.iloc[1, 3]))

https://www.dienmayxanh.com/dien-thoai/iphone-15-pro/danh-gia


In [12]:
# thử lấy số sao đánh giá và comment sản phẩm của một người dùng đánh giá
from selenium import webdriver

driver = webdriver.Chrome()
driver.get(str(uni_df.iloc[1, 3]))
html = driver.page_source
bs = BeautifulSoup(html, 'html.parser')
li_tags = bs.find_all('li', {'class': 'par'})
print(f'length li tags = {len(li_tags)}')
driver.quit()
for li in li_tags:
    stars = li.find_all('i', {'class': 'iconcmt-starbuy'})
    unstars = li.find_all('i', {'class': 'iconcmt-unstarbuy'})
    print(len(stars))
    print(len(unstars))
    comment = li.find('p', {'class': 'cmt-txt'})
    comment_txt = comment.get_text()
    print(comment_txt)
        
    # break

length li tags = 20
5
0
Máy dùng mượt, cầm vừa tay, màu titan tự nhiên rất đẹp, phân vân 15 pro max với 15 pro, nhưng khi cầm trên tay 15 pro max bị lớn quá...
4
1
Thái độ phục vụ tốt
3
2
15 pro của mình không thể tắt tự chỉnh độ sáng. Tắt True Tone nhưng độ sáng vẫn tự điều chỉnh. Đây là lỗi sản xuất hay tại sao nhỉ
3
2
Mau hết pin..gọi video call mau nóng
3
2
Đôi khi còn hơi lag
5
0
Cảm ơn hiện tại mấy vẫn rất mượt
5
0
Rất tốt
5
0
Rất tốt
5
0
Cho mình hỏi sao nhạc chuông ip15pro mặc định k giống như những dòng khác nhỉ
4
1
ok rất cảm ơn
4
1
Mơi mua ở tgdd nữa tháng xài thì ok nhưng sao sạc nóng máy quá
5
0
Máy xài rất OK, pin tốt, nhân viên nhiệt tình 10 điểm
4
1
Mới mua 1 sài rất ok nhưng tại sao sài 1 lúc là dt nóng quá
5
0
Cảm ơn cực tốt
5
0
Màu titan đẹp .cảm giác cầm nắm thoải mái .mọi tác vụ rất mượt với e chip a17 pro
5
0
Xài tạm dc để một thời gian nữa xem sao
5
0
K
5
0
Nhân viên quá tuyệt vời máy thì cần trải nghiệm thêm
5
0
máy sử dụng rất OK , mượt mà, chạy nhanh, màn hình

In [19]:
from selenium import webdriver


def vote(stars):
    if stars <= 2 and stars >= 0:
        return -1 # negative vote
    elif stars == 3:
        return 0 # neutral vote
    elif stars >= 4 and stars <= 5:
        return 1 # positive vote
    else:
        return 1000
    

file_path = "E:/UDA_LEARNING/UDA_XU LI NGON NGU TU NHIEN II/DienMayXanhCrawling/data/getCommentProduct_PhoneDevices.csv"
csv_file = open(file_path, 'a', newline='', encoding='utf-8')
writer = csv.writer(csv_file)

columns = ['data_id', 'brand', 'price', 'rating', 'href', 'stars', 'comment', 'target']
writer.writerow(columns)

driver = webdriver.Chrome()
# count = 0

for idx in range(0, len(uni_df)):
    # if count == 5:
    #     break
    # else:
    #     count += 1
    # meta data
    data_id = uni_df.iloc[idx, 0]
    brand = uni_df.iloc[idx, 7]
    price = uni_df.iloc[idx, 6]
    rating = uni_df.iloc[idx, 9]
    href = uni_df.iloc[idx, 3]
    # stars = 0
    # comment = ''
    # target = 1000

    
    # make sure that href is exist
    try:
        driver.get(str(href))
        time.sleep(random.randint(0, 7))
        html = driver.page_source
        bs = BeautifulSoup(html, 'html.parser')
        # state = 0: it's okay
        state = True
    except Exception as e:
        print(f'Error at idx={idx}, TypeError={e}')
        # state = -1: some error
        state = False
    
    # get stars and comments
    if state == True:
        # danh sách các comment được tìm thấy, mỗi comment đánh giá sản phẩm sẽ được bao trong một thẻ li
        li_tags = bs.find_all('li', {'class': 'par'})
        try:
            # trong mỗi comment đánh giá (của thẻ li), ta sẽ lấy số sao được đánh giá và comment của đánh giá trong thẻ li đó
            for li in li_tags:
                stars = len(li.find_all('i', {'class': 'iconcmt-starbuy'}))
                # unstars = len(li.find_all('i', {'class': 'iconcmt-unstarbuy'}))
                comment = li.find('p', {'class': 'cmt-txt'})
                target = vote(stars=stars)                
                comment_txt = comment.get_text()                
                # data to write a row on csv file
                meta_data = [data_id, brand, price, rating, href, stars, comment_txt, target]
                writer.writerow(meta_data)
        except Exception as e:
            print(e)
    else:
        print('state False at idx ', idx)
    time.sleep(random.randint(0, 7))

driver.quit()
csv_file.close()

In [21]:
file_path = "E:/UDA_LEARNING/UDA_XU LI NGON NGU TU NHIEN II/DienMayXanhCrawling/data/getCommentProduct_PhoneDevices.csv"
df = pd.read_csv(file_path)
df.head()


,data_id,brand,price,rating,href,stars,comment,target;
0,305659,iPhone (Apple),37490000.0,24,https://www.dienmayxanh.com/dien-thoai/iphone-...,5,Oke,1;
1,305659,iPhone (Apple),37490000.0,24,https://www.dienmayxanh.com/dien-thoai/iphone-...,5,Tốt,1;
2,"305659,iPhone (Apple) ,37490000.0,24,https://w...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"305659,iPhone (Apple) ,37490000.0,24,https://w...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,305659,iPhone (Apple),37490000.0,24,https://www.dienmayxanh.com/dien-thoai/iphone-...,1,Mới mua. Ngày 28/02. Sài màng hình đơ đơ cái c...,-1;
